The [Containers](Containers.ipynb) Tutorial introduced the [HoloMap](Containers.ipynb#HoloMap), a core HoloViews data structure that allows easy exploration of parameter spaces. The essence of a HoloMap is that it contains a collection of [Elements](Elements.ipynb) (e.g. Images and Curves) that you can easily select and visualize.

HoloMaps hold Elements at sampled points in a multidimensional space. Although this property makes them useful for exploring high-dimensional parameter spaces, HoloMaps can very quickly consume huge amounts of memory for such spaces. For instance, a hundred samples along four orthogonal dimensions would need a HoloMap containing a hundred *million* Elements, each of which could be a substantial object. Thus ``HoloMaps`` have some clear limitations:

* HoloMaps may require the generation of millions of elements before they are fully defined.
* HoloMaps can easily exhaust all the memory available to Python.
* HoloMaps can simultaneously exhaust all the memory in the browser when displayed.
* Static export of a notebook containing HoloMaps can result in impractically large HTML files.

The ``DynamicMap`` addresses these issues by computing and displaying elements dynamically, allowing exploration of much larger datasets:

* DynamicMaps generate elements on the fly allowing the process of exploration to begin immediately.
* DynamicMaps do not require fixed sampling, allowing exploration of parameters with arbitrary resolution.
* DynamicMaps are lazy in the sense they only compute only as much data as the user wishes to explore.

The limitations of ``DynamicMaps`` are:

* DynamicMaps require a live notebook server and cannot be directly exported to static HTML.
* DynamicMaps do not hold data which reduces the utility of a pickling DynamicMap.
* DynamicMaps are often stateful as are their element caches.

Now we have enumerated the pros and cons of DynamicMaps, let's create one!

<center><div class="alert alert-info" role="alert">To use visualize and use a <b>DynamicMap</b> you need to be running a live Jupyter server.<br>This tutorial assumes that it will be run in a live notebook environment.<br>
When viewed statically, DynamicMaps will only show the first available Element.</div></center>

## ``DynamicMap``  <a id='DynamicMap'></a>

Let's start by importing HoloViews and loading the extension:

In [ ]:
import holoviews as hv
import numpy as np
hv.notebook_extension()

We will now create the ``DynamicMap`` equivalent of the ``HoloMap`` introduced in the [Containers Tutorial](Containers.ipynb#HoloMap). The ``HoloMap`` in that tutorial consisted of ``Image`` elements containing sine ring arrays as defined by the ``sine_array`` function:

In [ ]:
x,y = np.mgrid[-50:51, -50:51] * 0.1

def sine_array(phase, freq):
    return np.sin(phase + (freq*x**2+freq*y**2))

This function returns NumPy arrays when called:

In [ ]:
sine_array(0,1).shape

For a ``DynamicMap`` we will need a function that returns HoloViews elements. This is easy as a trivial extension of the ``sine_array`` function allows it to output [Image](Elements.ipynb#Image) elements:

In [ ]:
def sine_image(phase, freq):
    return hv.Image(np.sin(phase + (freq*x**2+freq*y**2)))

sine_image(0,1) + sine_image(0.5,2)

Now we can demonstrate the first type of exploration enabled by a ``DynamicMap`` we call 'bounded' mode.

### Bounded mode

A 'bounded' mode ``DynamicMap`` is simply one where all the key dimensions have finite bounds. Bounded mode has the following properties:

* The limits of the space and/or the allowable values must be declared for all the key dimensions (unless [sampled mode](#SampledMode) is enabled).
* You can explore within the declared bounds at any resolution.
* The ``DynamicMap`` is defined using a callable that *must* be a function of its arguments (i.e., the output is strictly determined by the input arguments).

We can now create a DynamicMap by simply declaring the ranges of the two dimensions and passing the ``sine_image`` function as the ``.data``:



In [ ]:
dmap = hv.DynamicMap(sine_image, kdims=[hv.Dimension('phase',range=(0, np.pi)),
                                 hv.Dimension('frequency', range=(0.01,np.pi))])

This object is created instantly as no data has been generated yet. We can now look at the ``repr`` of this object: 

In [ ]:
repr(dmap)

All ``DynamicMaps`` will look similar, only differing in the listed dimensions. Now let's see how this dynamic map visualizes itself:

In [ ]:
dmap

You can combine ``DynamicMaps`` with each other (and any other HoloViews Element) to create a ``Layout`` using the ``+`` operator:

In [ ]:
dmap + hv.DynamicMap(sine_image, kdims=[hv.Dimension('phase',range=(0, np.pi)),
                                 hv.Dimension('frequency', range=(0.01,np.pi))])

As both elements are ``DynamicMaps`` with the same dimension ranges, the continuous sliders are retained. If one or more ``HoloMaps`` is used with a ``DynamicMap``, the sliders will snap to the samples available in any ``HoloMap`` in the layout. For bounded ``DynamicMaps`` that do not require ranges to be declared, see [sampled mode](#SampledMode).

If you are running this tutorial in a live notebook, you should now see something that looks like the ``HoloMap`` in the [Containers Tutorial](Containers.ipynb#HoloMap). ``DynamicMap`` is in fact a subclass of ``HoloMap`` with some crucial differences:

* You can now pick any value of **phase** or **frequency** up to the precision allowed by the slider.
* What you see in the cell above will not be exported in any HTML snapshot of the notebook.

#### Using your own callable

You can use any callable to define a ``DynamicMap`` in closed mode. A valid ``DynamicMap`` is defined by the following criteria:

* There must be as many positional arguments in the callable signature as key dimensions.
* The argument order in the callable signature must match the order of the declared key dimensions.
* All key dimensions are defined with a bounded ``range`` or ``values`` parameter (for categorical dimensions).

#### The ``DynamicMap`` cache

Above we mentioned that ``DynamicMap`` is an instance of ``HoloMap``. Does this means it has a ``.data`` attribute?

In [ ]:
dmap.data

This is exactly the same sort of ``.data`` as the equivalent ``HoloMap``, except that this value will vary according to how much you explored the parameter space of ``dmap`` using the sliders above. In a ``HoloMap``, ``.data`` contains a defined sampling along the different dimensions, whereas in a ``DynamicMap``, the ``.data`` is simply the *cache*.

The cache serves two purposes:

* Avoids recomputation of an element should we revisit a particular point in the parameter space. This works well for categorical data, but doesn't help much when using continuous sliders.
* Records the space that has been explored with the ``DynamicMap`` when converting to a ``HoloMap``.
* Ensures a finite history of generator output when using [open mode](#OpenMode) together with infinite generators.

We can always convert *any* ``DynamicMap`` directly to a ``HoloMap`` as follows:

In [ ]:
hv.HoloMap(dmap)

This is in fact equivalent to declaring a HoloMap with the same parameters using ``dmap.data`` as input.

Although creating a HoloMap this way is easy, the result is poorly controlled as the keys in the HoloMap are defined by how you moved the sliders around. This can be easily rectified using the same key selection semantics of ``HoloMap`` to define exactly which elements are sampled in the cache:

In [ ]:
dmap[{(0,0.01), (0,0.5), (0.5,0.01), (0.5,0.5)}] # Returns a *new* DynamicMap with the sampled keys in it's cache

This object behaves the same way as before it was sampled but now this ``DynamicMap`` can now be exported to static HTML with the allowed slider positions as specified in the cache (without even having to cast to a ``HoloMap``). Nonetheless, casting to a ``HoloMap`` makes the intention to generate a statically exportable, sampled visualization obvious.

As the key selection above defines a cartesian product which is one of the most common way to sample across dimensions, there is another convenient syntax which also works with ``HoloMaps``. Here is an equivalent way of defining the same set of keys:

In [ ]:
dmap[{0,0.5},{0.01,0.5}]

In [ ]:
dmap.data

Note that you can index a ``DynamicMap`` with a literal key in exactly the same way as a ``HoloMap``, so long as you use an exact key value that already exists in the cache:

In [ ]:
dmap[dmap.keys()[-1]]

The default cache size is the reasonably high value of 500 Elements. You can set the size using the ``cache_size`` parameter if you find you are running into issues with memory consumption. A bounded ``DynamicMap`` with ``cache_size=1`` requires the least memory, but will recompute a new Element every time the sliders are moved.

####  Slicing bounded ``DynamicMaps``

The declared dimension ranges define the absolute limits allowed for exploration in a bounded ``DynamicMap``. That said, you can use the ``soft_range`` parameter to view subregions within that range. This setting of the ``soft_range`` parameter on dimensions defines the semantics of slicing on bounded ``DynamicMaps``:

In [ ]:
sliced = dmap[0.2:0.8, :]
sliced

Open slices are used to release any ``soft_range`` values which resets the limits back to those defined by range:

In [ ]:
sliced[:, 0.05:0.1]

The ``[:]`` slice leaves the soft_range values alone and can be used as a convenient way to clone a ``DynamicMap``. Note that you are not allowed to mix slices when any other object type. In other words, once you use a single slice, you have to only uses slices in that indexing operation.

### Sampling DynamicMaps

We have now seen one way of sampling a ``DynamicMap``, which is to populate the cache with a set of keys. This approach is designed to make conversion of a ``DynamicMap`` into a ``HoloMap`` easy. One disadvantage of this type of sampling is that populating the cache consumes memory, resulting in many of the same limitations as ``HoloMap``. To avoid this, there are two other ways of sampling a bounded ``DynamicMap``:

#### Dimension values

If you want a fixed sampling instead of continuous sliders, yet still wish to retain the online generation of elements as the sliders are moved, you can simply set the dimension values. The result appears to the user just like a ``HoloMap``, and has the same data as the pre-cached version above, but now generates the data dynamically:

In [ ]:
hv.DynamicMap(sine_image, kdims=[hv.Dimension('phase',values=[0,0.5]),
                                 hv.Dimension('frequency',  values=[0.01,0.5])])

#### Sampled mode  <a id='SampledMode'></a>

A bounded ``DynamicMap`` in sampled mode is the least restricted type of ``DynamicMap``, as it can be declared without any information about the allowable dimension ranges or values:

In [ ]:
dmap = hv.DynamicMap(sine_image, kdims=['phase', 'frequency'], sampled=True)
dmap

As you can see, this type of ``DynamicMap`` cannot be visualized in isolation. As before, you can sample the cache and thereby  cast it to a HoloMap as necessary:

In [ ]:
dmap[{0,0.5},{0.01,0.5}]

The important usage of a sampled ``DynamicMap`` is how it interacts with ``HoloMaps`` in a Layout. As a sampled ``DynamicMap`` doesn't have explicitly declared dimension ranges, it can always adopt the set of sample values from ``HoloMaps`` in the layout.

In [ ]:
dmap + hv.HoloMap({(p,f):sine_image(p, f) 
                   for p in [0,0.5,1,1.5] 
                   for f in [0.5,0.75]}, kdims=['phase', 'frequency'])

The additional convenience of sampled ``DynamicMaps`` is subject to three particular restrictions:

* Sampled ``DynamicMaps`` do not visualize themselves in isolation (as we have already seen).
* You cannot build a layout consisting of ``DynamicMaps`` only as at least one HoloMap is needed to define the samples.
* There cannot be more dimensions declared in the sampled ``DynamicMap`` than across the rest of the layout. We hope to relax this restriction in future.

### Open mode <a id='OpenMode'></a>

``DynamicMap`` also allows unbounded exploration with unbounded dimensions in 'open' mode. There are two key differences of this mode with bounded mode:

* Instead of a callable, the input to an open ``DynamicMap`` is a generator. Once created, the generator is only used via ``next()``.
* At least one of the declared key dimensions must have an unbounded range. (upper range only? need to check!)
* An open mode ``DynamicMap`` can run forever, or until a ``StopIteration`` exception is raised.
* Open mode ``DynamicMaps`` can be stateful, with an irreversible direction of time.


#### Infinite generators

Our first example will be using an infinite generator, using the fact that phase is an infinite cyclic quantity:

In [ ]:
def sine_gen(phase=0, freq=0.5):
    while True:
        yield hv.Image(np.sin(phase + (freq*x**2+freq*y**2)))
        phase+=0.2

Just to confirm this is a generator:

In [ ]:
sine_gen()

And now for our infinite ``DynamicMap``:

In [ ]:
dmap = hv.DynamicMap(sine_gen(), kdims=['phase'])
dmap

Note that phase is shown as an integer. This is the default behavior and corresponds to the call count (i.e the number of times ``next()`` has been called on the generator. If we want to show the phase value properly, we need our generator to return a (key, element) pair:

In [ ]:
def sine_kv_gen(phase=0, freq=0.5):
    while True:
        yield (phase, hv.Image(np.sin(phase + (freq*x**2+freq*y**2))))
        phase+=0.2
        
hv.DynamicMap(sine_kv_gen(), kdims=['phase'])

Note that if you pause the ``DynamicMap``, you can scrub back to previous frames in the cache. In other words, you can view a limited history of elements already output by the generator, which does *not* re-execute the generator in any way (as it is indeed impossible to rewind generator state). If you have a stateful generator that, say, depends on the current wind speed in Scotland, this history may be misleading, in which case you can simply set the ``cache_size`` parameter to 1.

#### Multi-dimensional generators

In open mode, elements are naturally serialized by a linear sequence of ``next()`` calls. This doesn't mean that they can't have multiple key dimensions:

In [ ]:
def sine_kv_gen_2D(phase=0, freq=0.5):
    while True:
        yield ((phase, freq), hv.Image(np.sin(phase + (freq*x**2+freq*y**2))))
        phase+=0.2
        freq += 0.01
        
hv.DynamicMap(sine_kv_gen_2D(), kdims=['phase', 'frequency'])

#### Finite generators

Open mode ``DynamicMaps`` are finite and terminate if ``StopIteration`` is raised. This example uses a generator expression that is identical to the first open ``DynamicMap`` example above, except that it terminates after 20 phases:


In [ ]:
dmap=hv.DynamicMap((hv.Image(np.sin(0.2*phase + (0.5*x**2+0.5*y**2))) for phase in range(21)), kdims=['phase'])
dmap

It is trivial to adapt this generator expression to return (key, tuple) pairs, where the key may have one or more dimensions.

Now if you are familiar with generators in Python, you might be wondering what happens when a finite generator is exhausted. First we should mention that casting a ``DynamicMap`` to a list is always finite, because  ``__iter__`` returns the cache instead of a potentially infinite generator:

In [ ]:
list(dmap) # The cache

As we know this ``DynamicMap`` is finite, we can exhaust it as follows:

In [ ]:
while True:
    try:
        next(dmap) # Returns Image elements
    except StopIteration:
        print("The dynamic map is exhausted.")
        break

Now let's have a look at the dynamic map:

In [ ]:
dmap

We are given only the repr to reflect that the generator is exhausted. However, as the process of iteration has populated the cache, we can still view the output as a ``HoloMap`` using ``hv.HoloMap(dmap)`` as before.

### Counter mode and temporal state

Open mode is intended to interface live data streams or ongoing simulations with HoloViews. The ``DynamicMap`` will generate live visualizations for as long as new data is requested. Although this works for simple cases, Python generators have problematic limitations that can be resolved using 'counter' mode.

In this example, lets say we have a simulation or data recording where time increases in integer steps:

In [ ]:
def time_gen(time=1):
    while True:
        yield time
        time += 1
        
time = time_gen()

Now we have two generators that return Images that are a function of the simulation time. Here, they have identical output except one of the outputs includes additive noise:

In [ ]:
ls = np.linspace(0, 10, 200)
xx, yy = np.meshgrid(ls, ls)

def cells():
    while True:
        t = time.next()
        arr = np.sin(xx+t)*np.cos(yy+t)
        yield hv.Image(arr)

def cells_noisy():
    while True:
        t = time.next()
        arr = np.sin(xx+t)*np.cos(yy+t)
        yield hv.Image(arr + 0.2*np.random.rand(200,200))

Now let's create a Layout using these two generators:

In [ ]:
hv.DynamicMap(cells(), kdims=['time']) + hv.DynamicMap(cells_noisy(), kdims=['time'])

If you pause the animation, you'll see that these two outputs are *not* in phase, despite the fact that the generators are defined identically (modulo the additive noise)!

The issue is that generators are used via the ``next()`` interface, and so when either generator is called, the simulation time is increased. In other words, the noisy version in subfigure **B** actually corresponds to a later time than in subfigure **A**.

This is a fundamental issue, as the ``next`` method does not take arguments. What we want is for all the ``DynamicMaps`` presented in a Layout to share a common simulation time, which is only incremented by interaction with the scrubber widget. This is exactly the sort of situation where you want to use counter mode.

### Handling time-dependent state

To define a ``DynamicMap`` in counter mode:

* Leave one or more dimensions *unbounded* (as in open mode)
* Supply a callable (as in bounded mode) that accepts *one* argument

This callable should act in the same way as the generators of open mode except the output in controlled by the single counter argument.

In [ ]:
ls = np.linspace(0, 10, 200)
xx, yy = np.meshgrid(ls, ls)

def cells_counter(t):
    while True:
        arr = np.sin(xx+t)*np.cos(yy+t)
        return hv.Image(arr)

def cells_noisy_counter(t):
    while True:
        arr = np.sin(xx+t)*np.cos(yy+t)
        return hv.Image(arr + 0.2*np.random.rand(200,200))

Now if we supply these functions instead of generators:

In [ ]:
hv.DynamicMap(cells_counter, kdims=['time']) + hv.DynamicMap(cells_noisy_counter, kdims=['time'])

Now **A** and **B** are correctly in phase.

Unfortunately, an integer counter is often too simple to describe simulation time, which is more often a float with real-world units. To address this, we can simply return the actual key values we want along the time dimension, just as was demonstrated in open mode using generators:

In [ ]:
ls = np.linspace(0, 10, 200)
xx, yy = np.meshgrid(ls, ls)

# Example of a global simulation time
# typical in many applications
t = 0 
        
def cells_counter_kv(c):
    global t
    t = 0.1 * c
    while True:
        arr = np.sin(xx+t)*np.cos(yy+t)
        return (t, hv.Image(arr))

def cells_noisy_counter_kv(c):
    global t
    t = 0.1 * c
    while True:
        arr = np.sin(xx+t)*np.cos(yy+t)
        return (t, hv.Image(arr + 0.2*np.random.rand(200,200)))
    
hv.DynamicMap(cells_counter_kv, kdims=['time']) + hv.DynamicMap(cells_noisy_counter_kv, kdims=['time'])

In [ ]:
print("The global simulation time is now t=%f" % t)

Ensuring that the HoloViews counter maps to a suitable simulation time is the responsibility of the user. However, once a consistent scheme is configured, the callable in each ``DynamicMap`` can specify the desired simulation time. If the requested simulation time is the same as the current simulation time, nothing needs to happen. Otherwise, the simulator can be run forward by the requested amount.

### Slicing in open and counter mode

Slicing open and counter mode ``DynamicMaps`` has the exact same semantics as normal ``HoloMap`` slicing, except now the ``.data`` attribute corresponds to the cache. For instance:

In [ ]:
def sine_kv_gen(phase=0, freq=0.5):
    while True:
        yield (phase, hv.Image(np.sin(phase + (freq*x**2+freq*y**2))))
        phase+=0.2
        
dmap = hv.DynamicMap(sine_kv_gen(), kdims=['phase'])

Let's fill the cache with some elements:

In [ ]:
for i in range(21):
    dmap.next()
    
print("Min key value in cache:%s\nMax key value in cache:%s" % (min(dmap.keys()), max(dmap.keys())))

In [ ]:
sliced = dmap[1:3.1]
print("Min key value in cache:%s\nMax key value in cache:%s" % (min(sliced.keys()), max(sliced.keys())))

### Using DynamicMaps in your code

As you can see, ``DynamicMaps`` let you use HoloViews with a very wide range of dynamic data formats and sources, making it simple to visualize ongoing processes or very large data spaces.